# <center> <u> Feature Selection </u> </center>

<h2>What's the Purpose of Feature Selection</h2>
<p>Many learning algorithms perform poorly on high-dimensional data. This is known as the <b>curse of dimensionality</b>
    <p>There are other reasons we may wish to reduce the number of features including:
        <p>1. Reducing computational cost
            <p>2. Reducing the cost associated with data collection
                <p>3. Improving Interpretability

In [1]:
#import these libraries as we are going to use them. 
# Note: just have a look what all libraries you have imported
import pandas as pd
import numpy as np

from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LinearRegression
from math import sqrt

## 1.Filter Methods:



Filter method applies a statistical measure to assign a scoring to each feature.Then we can decide to keep or remove those features based on those scores. The methods are often univariate and consider the feature independently, or with regard to the dependent variable.

In this section we will cover below approaches:

1. Missing Value Ratio Threshold
2. Variance Threshold
3. $Chi^2$ Test
4. Anova Test

Download the dataset from https://github.com/arupbhunia/Data-Pre-processing/blob/master/datasets/diabetes.csv

Dataset used - diabetes.csv

Download instruction: go to the given link--->click raw button on top right corner---->Press Ctrl+S -->save it as .csv file.

## (a) Missing Value Ratio Threshold





In [ ]:
from google.colab import files
uploded = files.upload()

Saving diabetes.csv to diabetes.csv


In [ ]:
import io
# create a data frame named diabetes and load the csv file
diabetes = pd.read_csv(io.BytesIO(uploded['diabetes.csv']))
#print the head 
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


We know that some features can not be zero(e.g. a person's blood pressure can not be 0) hence we will impute zeros with nan value in these features.


In [ ]:
#Glucose BloodPressure, SkinThickness, Insulin, and BMI features cannot be zero ,we will impute zeros with nan value in these features.
col_to_fill = ["Glucose","BloodPressure", 'SkinThickness','Insulin','BMI']
for item in col_to_fill:
  diabetes[item].replace(0, np.NaN, inplace= True)


In [ ]:
#display the no of null values in each feature
diabetes.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

Now let's see for each feature what is the percentage of having missing values.

In [ ]:
diabetes.isnull().sum() * 100 / len(diabetes)

Pregnancies                  0.000000
Glucose                      0.651042
BloodPressure                4.557292
SkinThickness               29.557292
Insulin                     48.697917
BMI                          1.432292
DiabetesPedigreeFunction     0.000000
Age                          0.000000
Outcome                      0.000000
dtype: float64

In [ ]:
#percentage of missing values for Glucose
diabetes['Glucose'].isnull().sum() * 100 / len(diabetes['Glucose'])

0.6510416666666666

In [ ]:
# calculate the percentage for Bloodpressure
diabetes['BloodPressure'].isnull().sum() * 100 / len(diabetes['BloodPressure'])

4.557291666666667

In [ ]:
# calculate the percentage for SkinThickness
diabetes['SkinThickness'].isnull().sum() * 100 / len(diabetes['SkinThickness'])

29.557291666666668

In [ ]:
# calculate the percentage for Insulin
diabetes['Insulin'].isnull().sum() * 100 / len(diabetes['Insulin'])

48.697916666666664

In [ ]:
# calculate the percentage for BMI
diabetes['BMI'].isnull().sum() * 100 / len(diabetes['BMI'])

1.4322916666666667

Hey can you see that a large number of data missing in SkinThickness and Insulin.

Here we will keep only those features which are having missing data less than 10% as our threshold.


In [ ]:
# saving missing values in a variable
a = diabetes.isnull().sum()/len(diabetes)*100

# saving column names in a variable
variables = diabetes.columns

In [ ]:
# new variable to store variables having missing values less than a threshold

variable = [ ]

for i in range(diabetes.columns.shape[0]):
    if a[i]<=10: #setting the threshold as 10%
        variable.append(variables[i])

In [ ]:
# creating a new dataframe using the above variables

diabetes = diabetes[variable]
diabetes.shape

(768, 7)

In [ ]:
#we are keep only those features which are having missing data less than 10% 
diabetes_missing_value_threshold= .01

# print diabetes_missing_value_threshold
diabetes.dropna(thresh= diabetes_missing_value_threshold)

,Pregnancies,Glucose,BloodPressure,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,33.6,0.627,50,1
1,1,85.0,66.0,26.6,0.351,31,0
2,8,183.0,64.0,23.3,0.672,32,1
3,1,89.0,66.0,28.1,0.167,21,0
4,0,137.0,40.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...
763,10,101.0,76.0,32.9,0.171,63,0
764,2,122.0,70.0,36.8,0.340,27,0
765,5,121.0,72.0,26.2,0.245,30,0
766,1,126.0,60.0,30.1,0.349,47,1


Let's now Seperate the data diabetes_missing_value_threshold into features and labels 

Hey buddy! label is something which is dependent on other features for its outcome. You can also called it as our Target variable which we predict using ML algorithms.

Can you think which column would be considered as label.


In [ ]:

diabetes_missing_value_threshold_features = diabetes.iloc[:,:-1]

diabetes_missing_value_threshold_label= diabetes.iloc[:,-1]


In [ ]:
#print diabetes_missing_value_threshold_features
diabetes_missing_value_threshold_features

,Pregnancies,Glucose,BloodPressure,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,72.0,33.6,0.627,50
1,1,85.0,66.0,26.6,0.351,31
2,8,183.0,64.0,23.3,0.672,32
3,1,89.0,66.0,28.1,0.167,21
4,0,137.0,40.0,43.1,2.288,33
...,...,...,...,...,...,...
763,10,101.0,76.0,32.9,0.171,63
764,2,122.0,70.0,36.8,0.340,27
765,5,121.0,72.0,26.2,0.245,30
766,1,126.0,60.0,30.1,0.349,47


In [ ]:
#print diabetes_missing_value_threshold_label
diabetes_missing_value_threshold_label

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

## (b) Variance Threshold


If the variance is low or close to zero, then a feature is approximately constant and will not improve the performance of the model. In that case, it should be removed.

Variance will also be very low for a feature if only a handful of observations of that feature differ from a constant value.


Are you ready to implement feature selection using variance threshold? Smile and download the dataset diabetes_cleaned.csv from https://github.com/arupbhunia/Data-Pre-processing/blob/master/datasets/diabetes_cleaned.csv

Dataset used - diabetes_cleaned.csv

In [2]:
from google.colab import files
uploded = files.upload()

Saving diabetes_cleaned.csv to diabetes_cleaned.csv


In [3]:
import io
# load the csv to dataframe name "diabetes" and print the head values
diabetes = pd.read_csv(io.BytesIO(uploded['diabetes_cleaned.csv']))

# display diabetes.head()
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,218.937760,33.6,0.627,50.0,1
1,1.0,85.0,66.0,29.0,70.189298,26.6,0.351,31.0,0
2,8.0,183.0,64.0,29.0,269.968908,23.3,0.672,32.0,1
3,1.0,89.0,66.0,23.0,94.000000,28.1,0.167,21.0,0
4,0.0,137.0,40.0,35.0,168.000000,43.1,2.288,33.0,1


In [6]:
# seperate the features and the target as x and y 
x = diabetes.drop('Outcome', axis=1)
y = diabetes['Outcome']


In [7]:
print(type(x))
print("----------------------------")
print(type(y))


<class 'pandas.core.frame.DataFrame'>
----------------------------
<class 'pandas.core.series.Series'>


In [8]:
# Return  the variance for X along the specified axis=0.
print(diabetes.var())

Pregnancies                   11.354056
Glucose                      929.680350
BloodPressure                146.321591
SkinThickness                 77.285567
Insulin                     9484.259268
BMI                           48.813618
DiabetesPedigreeFunction       0.109779
Age                          138.303046
Outcome                        0.227483
dtype: float64


In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    float64
 1   Glucose                   768 non-null    float64
 2   BloodPressure             768 non-null    float64
 3   SkinThickness             768 non-null    float64
 4   Insulin                   768 non-null    float64
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    float64
dtypes: float64(8)
memory usage: 48.1 KB


In [10]:
# import minmax_scale
from sklearn.preprocessing import MinMaxScaler
# use minmax scale with feature_range=(0,10) and columns=X.columns,to scale the features of dataframe and store them into X_scaled_df 
scaler = MinMaxScaler(feature_range=(0,10))
diabetes[x.columns] = scaler.fit_transform(diabetes[x.columns])


Wait a minute! whats minmax scaling?

It is the simplest method and consists in rescaling the range of features to scale the range in [0, 1] or [−1, 1]

hey hey heyieeee! Fun fact time:

There is another scaling method called StandardScaler which follows Standard Normal Distribution (SND). Therefore, it makes mean = 0 and scales the data to unit variance. 

Cool right? :)

In [11]:
# return X_scaled_df
X_scaled_df = diabetes[x.columns]
X_scaled_df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,3.529412,6.709677,4.897959,3.043478,2.740295,3.149284,2.344150,4.833333
1,0.588235,2.645161,4.285714,2.391304,1.018185,1.717791,1.165670,1.666667
2,4.705882,8.967742,4.081633,2.391304,3.331099,1.042945,2.536294,1.833333
3,0.588235,2.903226,4.285714,1.739130,1.293850,2.024540,0.380017,0.000000
4,0.000000,6.000000,1.632653,3.043478,2.150572,5.092025,9.436379,2.000000
...,...,...,...,...,...,...,...,...
763,5.882353,3.677419,5.306122,4.456522,2.289500,3.006135,0.397096,7.000000
764,1.176471,5.032258,4.693878,2.173913,2.044244,3.803681,1.118702,1.000000
765,2.941176,4.967742,4.897959,1.739130,1.502241,1.635992,0.713066,1.500000
766,0.588235,5.290323,3.673469,2.391304,2.217956,2.433538,1.157131,4.333333


In [12]:
# Again return  the variance for X along the specified axis=0 to check the scales after using minmax scaler.
print(X_scaled_df.var())

Pregnancies                 3.928739
Glucose                     3.869637
BloodPressure               1.523548
SkinThickness               0.913109
Insulin                     1.271218
BMI                         2.041377
DiabetesPedigreeFunction    2.001447
Age                         3.841751
dtype: float64


In [13]:
# import variancethreshold
from sklearn.feature_selection import VarianceThreshold

# set threshold=1 and define it to variable select_features
select_features = VarianceThreshold(threshold = 1)

   Impliment fit_transform on select_features passing X_scaled_df into it and save this result in variable X_variance_threshold_df


In [14]:
X_variance_threshold_df= select_features.fit_transform(X_scaled_df)


Were you thinking of fit_transform? We are here to help you understand

fit_transform() is used on the training data so that we can scale the training data and also learn the scaling parameters of that data. Here, the model built by us will learn the mean and variance of the features of the training set. These learned parameters are then used to scale our test data

Don't worry you will get lot of challenges to use these things in our other assignments

In [15]:
#print X_variance_threshold_df
X_variance_threshold_df

array([[3.52941176, 6.70967742, 4.89795918, ..., 3.14928425, 2.3441503 ,
        4.83333333],
       [0.58823529, 2.64516129, 4.28571429, ..., 1.71779141, 1.16567037,
        1.66666667],
       [4.70588235, 8.96774194, 4.08163265, ..., 1.04294479, 2.53629377,
        1.83333333],
       ...,
       [2.94117647, 4.96774194, 4.89795918, ..., 1.63599182, 0.71306576,
        1.5       ],
       [0.58823529, 5.29032258, 3.67346939, ..., 2.43353783, 1.15713066,
        4.33333333],
       [0.58823529, 3.16129032, 4.69387755, ..., 2.49488753, 1.01195559,
        0.33333333]])

In [16]:
#Convert X_variance_threshold_df into dataframe
X_variance_threshold_df = pd.DataFrame(X_variance_threshold_df)

In [17]:
# print of head values of X_variance_threshold_df 
X_variance_threshold_df.head()

,0,1,2,3,4,5,6
0,3.529412,6.709677,4.897959,2.740295,3.149284,2.344150,4.833333
1,0.588235,2.645161,4.285714,1.018185,1.717791,1.165670,1.666667
2,4.705882,8.967742,4.081633,3.331099,1.042945,2.536294,1.833333
3,0.588235,2.903226,4.285714,1.293850,2.024540,0.380017,0.000000
4,0.000000,6.000000,1.632653,2.150572,5.092025,9.436379,2.000000


Below mentioned is the function get_selected_features for returning selected_features to be used further.


In [18]:
def get_selected_features(raw_df,processed_df):
    selected_features=[]
    for i in range(len(processed_df.columns)):
        for j in range(len(raw_df.columns)):
            if (processed_df.iloc[:,i].equals(raw_df.iloc[:,j])):
                selected_features.append(raw_df.columns[j])
    return selected_features

In [20]:
# pass the X_scaled_df as raw_df and X_variance_threshold_df as processed_df inside get_selected_features function
selected_features= get_selected_features(X_scaled_df,X_variance_threshold_df)

# print selected_features
print(selected_features)

['Pregnancies', 'Glucose', 'BloodPressure', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']


you can see SkinThickness feature is not selected as its variance is less.

Lets give column names to our X_variance_threshold_df

In [30]:
# define selected_features as columns and save it in variabe named X_variance_threshold_df
X_variance_threshold_df.columns = selected_features
#print X_variance_threshold_df
X_variance_threshold_df

,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age
0,3.529412,6.709677,4.897959,2.740295,3.149284,2.344150,4.833333
1,0.588235,2.645161,4.285714,1.018185,1.717791,1.165670,1.666667
2,4.705882,8.967742,4.081633,3.331099,1.042945,2.536294,1.833333
3,0.588235,2.903226,4.285714,1.293850,2.024540,0.380017,0.000000
4,0.000000,6.000000,1.632653,2.150572,5.092025,9.436379,2.000000
...,...,...,...,...,...,...,...
763,5.882353,3.677419,5.306122,2.289500,3.006135,0.397096,7.000000
764,1.176471,5.032258,4.693878,2.044244,3.803681,1.118702,1.000000
765,2.941176,4.967742,4.897959,1.502241,1.635992,0.713066,1.500000
766,0.588235,5.290323,3.673469,2.217956,2.433538,1.157131,4.333333


## (c) Chi-Squared statistical test (SelectKBest)



Chi2 is a measure of dependency between two variables. It gives us a goodness of fit measure because it measures how well an observed distribution of a particular feature fits with the distribution that is expected if two features are independent.

Note: Chi-Square is to be used when the feature is categorical, the target variable is any way can be thought as categorical. It measures the degree of association between two categorical variables. If both are numeric, we can use Pearson’s product-moment correlation, and if the attribute is numerical and there are two classes we can use a t-test if more than two classes we can use ANOVA.It may be noted Chi-Square can be used for the numerical variable as well after it is suitably discretized.


Scikit-Learn offers a feature selection estimator named SelectKBest which select K numbers of features based on the statistical analysis.


The below mentioned function generate_feature_scores_df is used to get feature score for using it in  Chi-Squared statistical test explained below

In [17]:
def generate_feature_scores_df(X,Score):
    feature_score=pd.DataFrame()
    for i in range(X.shape[1]):
        new =pd.DataFrame({"Features":X.columns[i],"Score":Score[i]},index=[i])
        feature_score=pd.concat([feature_score,new])
    return feature_score

Lets use dataset - diabetes.csv from below link

https://github.com/arupbhunia/Data-Pre-processing/blob/master/datasets/diabetes.csv

In [2]:
from google.colab import files
uploded = files.upload()
# create a data frame named diabetes and load the csv file again
import io
diabetes = pd.read_csv(io.BytesIO(uploded['diabetes.csv']))

Saving diabetes.csv to diabetes.csv


In [34]:
diabetes.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [3]:
# assign features to X variable and 'outcome' to y variable from the dataframe diabetes
X = diabetes.iloc[:, :-1]
Y = diabetes.iloc[:, -1]

In [4]:
#import chi2 and SelectKBest

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [5]:
# converting data cast to a float type.
X = X.astype(float)

In [81]:
print(type(X))

<class 'pandas.core.frame.DataFrame'>


Lets use SelectKBest to calculate the best feature score. Use Chi2 as Score Function and no.of feature i.e. k as 4


In [82]:
# Initialise SelectKBest with above parameters 
chi2_test= SelectKBest(chi2, k=4)

# fit it with X and Y
chi2_model= chi2_test.fit(X, Y)



In [83]:
#print the scores of chi2_model
chi2_model.scores_

array([ 111.51969064, 1411.88704064,   17.60537322,   53.10803984,
       2175.56527292,  127.66934333,    5.39268155,  181.30368904])

In [ ]:
feature_score_df = 

In [84]:
# use generate_feature_scores_df function to get features and their respective scores passing X and chi2_model.scores_ as paramter
feature_score_df= generate_feature_scores_df(X,chi2_model.scores_)


# return feature_score_df
feature_score_df


,Features,Score
0,Pregnancies,111.519691
1,Glucose,1411.887041
2,BloodPressure,17.605373
3,SkinThickness,53.108040
4,Insulin,2175.565273
5,BMI,127.669343
6,DiabetesPedigreeFunction,5.392682
7,Age,181.303689


Did you see the features and corresponding chi square scores? This is so easy right, higher the score better the feature. Just like higher the marks in assignment better the student of ours. 

In [85]:
#Lets get X with selected features of chi2_model using tranform function so we will have X_new
X_new = chi2_test.transform(X)


whaaaat! tranform()? , how is it different from fit_transform.You know buddy fit() can also confuse you. Hey you inquisitive learner we will tell you the difference.

The fit() function calculates the values of these parameters. The transform function applies the values of the parameters on the actual data and gives the normalized value. The fit_transform() function performs both in the same step. Note that the same value is got whether we perform in 2 steps or in a single step.


In [86]:
print(type(X_new))

<class 'numpy.ndarray'>


In [87]:
# Convert X_new into a dataframe

X_new= pd.DataFrame(X_new)

In [89]:
#repeat the previous steps of calling get_selected_features function( pass X and X_new as score in the function)
selected_features= get_selected_features(X,X_new)

# return selected_features
selected_features

['Glucose', 'Insulin', 'BMI', 'Age']

Let have X with all features given in list selected_features and save this dataframe in variable chi2_best_features

In [90]:
chi2_best_features= X.loc[:,['Glucose', 'Insulin', 'BMI', 'Age']]

# print chi2_best_features.head()
chi2_best_features.head()

,Glucose,Insulin,BMI,Age
0,148.0,0.0,33.6,50.0
1,85.0,0.0,26.6,31.0
2,183.0,0.0,23.3,32.0
3,89.0,94.0,28.1,21.0
4,137.0,168.0,43.1,33.0


As you can see chi-squared test helps us to select  important independent features out of the original features that have the strongest relationship with the target feature.

You did it well!

As you can see chi-squared test helps us to select  important independent features out of the original features that have the strongest relationship with the target feature.

You did it well!

Now many learners have this confusion below on two questions frequently on this section:-
1. How can the χ2-test work for feature selection for continuous variables here (since Glucose, Insulin, BMI and Age are continues and only target is categorical)?
2. Is it a problem that this test is scale dependent?



Mostly we get confused on the data itself (which can be continuous) with the fact that when you talk about data, you actually talk about samples, which are discrete.

The χ2 test (in wikipedia and the model selection by χ2 criterion) is a test to check for independence of sampled data. I.e. when you have two (or more) of sources of the data (i.e. different features), and you want to select only features that are mutually independent, you can test it by rejecting the Null hypothesis (i.e. data samples are dependant) if the probability to encounter such a sample (under the Null hypothesis), i.e. the p value, is smaller than some threshold value (e.g., p < 0.05)

So now for above two questions,

1. The χ2 test do work only on categorical data, as you must count the occurences of the samples in each category to use it, but as I've mentioned above, when you use it, you actually have samples in hand, so one thing you can do is to divide your samples into categories based on thresholds (e.g., cat1:x∈[th1<x<th2],cat2:x∈[th2<x<th3], etc.) and count all the samples that fall into each category.
2. As for the scales - you are obviously must use the same scales when you discretize your samples, otherwise it won't make any sense, but when you conduct the χ2 test itself, you are dealing with counts, so they won't have any scales anyway.


Hope this clears your doubt. 

Hence you can see that in sklearn library also example of Chi2 test is given on numerical features i.e. iris data.

## (d) Anova-F Test



The F-value scores examine the varaiance by grouping the numerical feature by the target vector, the means for each group are significantly different.

In [92]:
#import libraries
from sklearn.feature_selection import f_classif,SelectPercentile


# Initialise SelectPercentile function with parameters f_classif and percentile as 80
Anova_test= SelectPercentile(f_classif, percentile = 80)



#Fit the above object to the features and target i.e X and Y
Anova_model= Anova_test.fit(X,Y)

In [93]:
# return scores of anova model
Anova_model.scores_

array([ 39.67022739, 213.16175218,   3.2569504 ,   4.30438091,
        13.28110753,  71.7720721 ,  23.8713002 ,  46.14061124])

In [94]:
# use generate_feature_scores_df function to get features and their respective scores by passing X and Anova_model.scores_ as score in function 

feature_scores_df= generate_feature_scores_df(X,Anova_model.scores_)

# print feature_scores_df
feature_score_df

,Features,Score
0,Pregnancies,111.519691
1,Glucose,1411.887041
2,BloodPressure,17.605373
3,SkinThickness,53.108040
4,Insulin,2175.565273
5,BMI,127.669343
6,DiabetesPedigreeFunction,5.392682
7,Age,181.303689


In [95]:
# Get all supported columns values in Anova_model with indices=True
cols = Anova_model.get_support(indices= True)
# Reduce X to the selected features of anova model using tranform 

X_new = X.iloc[:,cols]



In [96]:
#print X_new.head()
X_new.head()

,Pregnancies,Glucose,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6.0,148.0,0.0,33.6,0.627,50.0
1,1.0,85.0,0.0,26.6,0.351,31.0
2,8.0,183.0,0.0,23.3,0.672,32.0
3,1.0,89.0,94.0,28.1,0.167,21.0
4,0.0,137.0,168.0,43.1,2.288,33.0


# 2. Wrapper Methods



Wrapper methods are used to select a set of features by preparing where different combinations of features, then each combination is evaluated and compared to other combinations.Next a predictive model is used to assign a score based on model accuracy and to evaluate the combinations of these features.

In [6]:
# load and read the diabetes.csv using pandas and print the head values
uploded = files.upload()

df = pd.read_csv(io.BytesIO(uploded['diabetes.csv']))
df.head()

Saving diabetes.csv to diabetes (1).csv


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
# assign features to X and target 'outcome' to Y variable(Think why the 'outcome' column is taken as the target)
X= df.iloc[:,:-1]
Y= df.iloc[:,-1]

#return X,Y
print(X)
print(Y)

     Pregnancies  Glucose  BloodPressure  ...   BMI  DiabetesPedigreeFunction  Age
0              6      148             72  ...  33.6                     0.627   50
1              1       85             66  ...  26.6                     0.351   31
2              8      183             64  ...  23.3                     0.672   32
3              1       89             66  ...  28.1                     0.167   21
4              0      137             40  ...  43.1                     2.288   33
..           ...      ...            ...  ...   ...                       ...  ...
763           10      101             76  ...  32.9                     0.171   63
764            2      122             70  ...  36.8                     0.340   27
765            5      121             72  ...  26.2                     0.245   30
766            1      126             60  ...  30.1                     0.349   47
767            1       93             70  ...  30.4                     0.315   23

[76

## (a) Recursive Feature Elemination


Recursive Feature Elimination selects features by recursively considering smaller subsets of features by pruning the least important feature at each step. Here models are created iteartively and in each iteration it determines the best and worst performing features and this process continues until all the features are explored.Next ranking is given on eah feature based on their elimination orde. In the worst case, if a dataset contains N number of features RFE will do a greedy search for $N^2$ combinations of features.

In [8]:
# import RFE and LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression



In [9]:
# Initialise model variable with LogisticRegression function with solver = 'liblinear'
model= LogisticRegression(solver= 'liblinear')

# rfe variable has RFE instance with should have model and n_features_to_select=4 as parameters

rfe= RFE(model, n_features_to_select=4)

In [10]:
# fit rfe with X and Y

fit= rfe.fit(X,Y)



In [11]:
print('Number of selected features',fit.n_features_)
print('Selected Features',fit.support_)
print('Feature rankings',fit.ranking_)

Number of selected features 4
Selected Features [ True  True False False False  True  True False]
Feature rankings [1 1 2 4 5 1 1 3]


In [12]:
# use below function to get ranks of all the features
def feature_ranks(X,Rank,Support):
    feature_rank=pd.DataFrame()
    for i in range(X.shape[1]):           # iteration through column
        new =pd.DataFrame({"Features":X.columns[i],"Rank":Rank[i],'Selected':Support[i]},index=[i])
        feature_rank=pd.concat([feature_rank,new])    # merging two dataframe
    return feature_rank


In [13]:
#Get all feature's ranks using feature_ranks function with suitable parameters. Sotre it in variable called feature_rank_df
feature_rank_df= feature_ranks(X,fit.ranking_, fit.support_)

# print feature_rank_df
feature_rank_df

,Features,Rank,Selected
0,Pregnancies,1,True
1,Glucose,1,True
2,BloodPressure,2,False
3,SkinThickness,4,False
4,Insulin,5,False
5,BMI,1,True
6,DiabetesPedigreeFunction,1,True
7,Age,3,False


We can see there are four features with rank 1 ,RFE states that these are the most significant features.

In [15]:
type(feature_rank_df.Selected)

pandas.core.series.Series

In [16]:
# filter feature_rank_df  with selected column values as True and save result in variable called recursive_feature_names 
recursive_feature_names= feature_rank_df[feature_rank_df['Rank'] ==1]

# print recursive_feature_names

recursive_feature_names

,Features,Rank,Selected
0,Pregnancies,1,True
1,Glucose,1,True
5,BMI,1,True
6,DiabetesPedigreeFunction,1,True


In [18]:
# finally get dataframe X with all the features selected by RFE and store this result in variable called RFE_selected_features
col = rfe.get_support(indices=True)
RFE_selected_features= X.iloc[:,col]
# print RFE head()

RFE_selected_features.head()

,Pregnancies,Glucose,BMI,DiabetesPedigreeFunction
0,6,148,33.6,0.627
1,1,85,26.6,0.351
2,8,183,23.3,0.672
3,1,89,28.1,0.167
4,0,137,43.1,2.288


# 3. Embedded Method using random forest



Feature selection using Random forest comes under the category of Embedded methods. Embedded methods combine the qualities of filter and wrapper methods. They are implemented by algorithms that have their own built-in feature selection methods. Some of the benefits of embedded methods are :
1. They are highly accurate.
2. They generalize better.
3. They are interpretable

In [19]:
#Importing libraries RandomForestClassifier and SelectFromModel
from sklearn.ensemble import  RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


In [20]:
uploaded = files.upload()
# load the csv file using pandas and print the head values
diab_df = pd.read_csv(io.BytesIO(uploaded['diabetes.csv']))
# print diabetes.head()
diab_df.head()

Saving diabetes.csv to diabetes (2).csv


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In all feature selection procedures, it is a good practice to select the features by examining only the training set. This is to avoid overfitting.
So considering we have a train and a test dataset. We select the features from the train set and then transfer the changes to the test set later

In [21]:
# assign features to X and target 'outcome' to Y(Think why the 'outcome' column is taken as the target)
X = diab_df.iloc[:,:-1]
Y = diab_df.iloc[:,-1]

In [26]:
# import test_train_split module
from sklearn.model_selection import train_test_split
# splitting of dataset(test_size=0.3)
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size= 0.3)

Here We will do the model fitting and feature selection altogether in one line of code.

Firstly, specify the random forest instance, indicating the number of trees.

Then use selectFromModel object from sklearn to automatically select the features.

In [27]:
#create an instance of Select from Model. Pass an object of Random Forest Classifier with n_estimators=100 as argument. 
sel = SelectFromModel(estimator=RandomForestClassifier(n_estimators=100))


# fit sel on training data
sel.fit(X_train,y_train)


SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight=None,
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=None,
                                                 oob_score=False,

SelectFromModel will select those features which importance is greater than the mean importance of all the features by default, but we can alter this threshold if we want.

 To see which features are important we can use get_support method on the fitted model.

In [28]:
# Using sel.get_support() print the boolean values for the features selected. 

sel.get_support()

array([False,  True, False, False, False,  True, False,  True])

In [36]:
col = sel.get_support(indices=True)
# RFE_selected_features= X.iloc[:,col]
#make a list named selected_feat with all columns which are True
selected_feat= list(X.iloc[:,col].columns)

# print length of selected_feat
len(selected_feat)

3

In [38]:
# Print selected_feat
print(selected_feat)

['Glucose', 'BMI', 'Age']


## Feature selection using SelectFromModel



SelectFromModel is a meta-transformer that can be used along with any estimator that has a coef_ or featureimportances attribute after fitting. The features are considered unimportant and removed, if the corresponding coef_ or featureimportances values are below the provided threshold parameter. Apart from specifying the threshold numerically, there are built-in heuristics for finding a threshold using a string argument. Available heuristics are “mean”, “median” and float multiples of these like “0.1*mean”.

Lets use selectfrommodel again with LinearSVC

In [39]:
# import LinearSVC 
from sklearn.svm import LinearSVC

In [40]:
#Use SelectFromModel with LinearSVC() as its parameter and save it in variable 'm'

m = SelectFromModel(LinearSVC())

#fit m with X and Y
m.fit(X,Y)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


SelectFromModel(estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                    fit_intercept=True, intercept_scaling=1,
                                    loss='squared_hinge', max_iter=1000,
                                    multi_class='ovr', penalty='l2',
                                    random_state=None, tol=0.0001, verbose=0),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [41]:
#make a list named selected_feat with all columns which are supported
col = m.get_support(indices=True)
selected_feat= list(X.iloc[:, col])

print(selected_feat)

['Pregnancies', 'DiabetesPedigreeFunction']


 # 4. Handling Multicollinearity with VIF
 



Multicollinearity refers to a situation in which more than two explanatory variables in a multiple regression model are highly linearly related. We have perfect multicollinearity if, for example as in the equation above, the correlation between two independent variables is equal to 1 or −1.

Variance inflation factor measures how much the behavior (variance) of an independent variable is influenced, or inflated, by its interaction/correlation with the other independent variables.

VIF has big defination but for now understand that:-
Variance inflation factor (VIF) is a measure of the amount of multicollinearity in a set of multiple regression variables

In [43]:
#load and read the diabetes_cleaned.csv file using pandas and print the head values
uploaded = files.upload()
dia_df= pd.read_csv(io.BytesIO(uploaded['diabetes.csv']))
dia_df.head()

Saving diabetes.csv to diabetes (3).csv


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [44]:
# describe the dataframe using .describe()
dia_df.describe()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


As we can see range of these features are very different that means they all are in different scales so lets standardize the features using sklearn's preprocessing scale function.


In [51]:
from sklearn import preprocessing

#iterate over all features in dia_df and scale
scaler = preprocessing.StandardScaler()
scaler.fit(dia_df)
df_scaled = scaler.transform(dia_df)

In [55]:
col = dia_df.columns

In [56]:
dia_df_csaled = pd.DataFrame(df_scaled, columns= col)

In [58]:
# describe dataframe using .describe()
dia_df_csaled.describe()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02
mean,2.544261e-17,3.614007e-18,-1.327244e-17,7.994184e-17,-3.556183e-17,2.295979e-16,2.398978e-16,1.857600e-16,2.408374e-16
std,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00
min,-1.141852e+00,-3.783654e+00,-3.572597e+00,-1.288212e+00,-6.928906e-01,-4.060474e+00,-1.189553e+00,-1.041549e+00,-7.321202e-01
25%,-8.448851e-01,-6.852363e-01,-3.673367e-01,-1.288212e+00,-6.928906e-01,-5.955785e-01,-6.889685e-01,-7.862862e-01,-7.321202e-01
50%,-2.509521e-01,-1.218877e-01,1.496408e-01,1.545332e-01,-4.280622e-01,9.419788e-04,-3.001282e-01,-3.608474e-01,-7.321202e-01
75%,6.399473e-01,6.057709e-01,5.632228e-01,7.190857e-01,4.120079e-01,5.847705e-01,4.662269e-01,6.602056e-01,1.365896e+00
max,3.906578e+00,2.444478e+00,2.734528e+00,4.921866e+00,6.652839e+00,4.455807e+00,5.883565e+00,4.063716e+00,1.365896e+00


In [59]:
#import variance inflation factor
from statsmodels.stats.outliers_influence import variance_inflation_factor
  

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [70]:
# assign features to X and target to Y 
X = dia_df.iloc[:,:-1]
Y = dia_df.iloc[:,-1]


# split the data to test and train with test_size=0.2
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size= 0.2)

In [61]:
#assign an empty dataframe to variable vif
vif= pd.DataFrame()

# make a new column 'VIF Factor' in vif dataframe and calculate the variance_inflation_factor for each X 
vif['VIF Factor']= [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

In [62]:
# define vif['Features'] with columns names in X

vif['Features']= X.columns

In [63]:
print(round(vif,2))

   VIF Factor                  Features
0        3.28               Pregnancies
1       16.73                   Glucose
2       14.62             BloodPressure
3        4.01             SkinThickness
4        2.06                   Insulin
5       18.41                       BMI
6        3.20  DiabetesPedigreeFunction
7       13.49                       Age


In [64]:
#  round off all the decimal values in the dataframe to 2 decimal places for VIF dataframe and print it.

print(round(vif,1))

   VIF Factor                  Features
0         3.3               Pregnancies
1        16.7                   Glucose
2        14.6             BloodPressure
3         4.0             SkinThickness
4         2.1                   Insulin
5        18.4                       BMI
6         3.2  DiabetesPedigreeFunction
7        13.5                       Age


* VIF = 1: Not correlated
* VIF =1-5: Moderately correlated
* VIF >5: Highly correlated

Glucose, Insulin, and Age are having large VIF scores, so lets drop it.



In [71]:
# according to above observation , drop  'Glucose', 'Insulin' and 'Age' from X

X.drop(['Glucose', 'Insulin','Age'], axis = 1,inplace= True)

Now again we calculate the VIF for the rest of the features

Again repeat the previous steps to assign an empty dataframe() to vif and make a new column 'VIF Factor' and calculate the variance_inflation_factorfor each X 


In [72]:
#code here
vif = pd.DataFrame()
vif['VIF Factor'] =  [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]


In [73]:
#define vif['Features'] as columns of X and return vif with round off to 2 decimal places
vif['Features'] =  X.columns
print(round(vif,2))


   VIF Factor                  Features
0        2.34               Pregnancies
1       11.84             BloodPressure
2        3.23             SkinThickness
3       13.71                       BMI
4        3.04  DiabetesPedigreeFunction


So now colinearity of features has been reduced using VIF.

The need to fix multicollinearity depends primarily on the below reasons:

1. When you care more about how much each individual feature rather than a group of features affects the target variable, then removing multicollinearity may be a good option
2. If multicollinearity is not present in the features you are interested in, then multicollinearity may not be a problem.

------------------------------